In [211]:
import numpy as np
from numpy.random import randint
from random import random as rnd
from random import gauss, randrange
import random
from datetime import datetime
import copy


In [212]:
path = "./me_at_the_zoo.in"
path = "./videos_worth_spreading.in"
#path = "./trending_today.in"
#path = "./kittens.in.txt"

In [213]:
def update_latency(a):
    difference_to_endpoints = {}
    endpoints_to_video = list_endpoints[a[0]]
    for endpoint in endpoints_to_video:
        difference_to_endpoints[endpoint] = request_dict[(endpoint, a[0])]*(a[2] if (endpoint,a[1]) in latency else 0)
    for c in lat_gain[a[0]]:
            reduction_in_lat_gain = sum([difference_to_endpoints[endpoint] for endpoint in endpoints_to_video if (endpoint, c) in latency])
            lat_gain[a[0]][c] = (lat_gain[a[0]][c]*sizes_videos[a[0]]-reduction_in_lat_gain)/sizes_videos[a[0]]
        

In [214]:
with open(path) as f:
    V, E, R, C, X = [int(x) for x in next(f).split()]
    sizes_videos = [int(x) for x in next(f).split()]
    endpoints = []
    for i in range(E):
        datacen_lat, num_caches = [int(x) for x in next(f).split()]
        connection = []
        for j in range(num_caches):
            ep_lat = [int(x) for x in next(f).split()]
            connection.append(ep_lat)
        endpoints.append([datacen_lat, num_caches, connection])
    requests = []
    for i in range(R):
        requests.append([int(x) for x in next(f).split()])

In [215]:
# Make a latency map

latency = {}
# (endpoint, cache) -> latency
for i in range(E):
    latency[(i,-1)] = endpoints[i][0]
    for j in range(endpoints[i][1]):
        latency[(i,endpoints[i][2][j][0])] = endpoints[i][2][j][1]

    

In [216]:
request_dict = {}
# (endpoint, video) -> num_request
for i in range(R):
    request_dict[(requests[i][1], requests[i][0])] = requests[i][2]

In [217]:
# video -> list of endpoints requesting it
list_endpoints = {}
for i in range(V):
    list_endpoints[i] = []
for i in range(R):
    list_endpoints[requests[i][0]].append(requests[i][1])

In [218]:
# Map for latency gain
lat_gain = {}
# (video)-> (cache) -> if added then what is gained latency at the moment
for v in range(V):
    lat_gain[v] = {}
    for c in range(C):
        original_latency = sum([(latency[(list_endpoints[v][x],-1)])*request_dict[(list_endpoints[v][x],v)] for x in range(len(list_endpoints[v]))])
        #latency[(2,0)]
        reduced_latency = sum([(latency[(list_endpoints[v][x], c)])*request_dict[(list_endpoints[v][x],v)] for x in range(len(list_endpoints[v])) if (list_endpoints[v][x],c) in latency])
        lat_gain[v][c] = (original_latency - reduced_latency)/sizes_videos[v]

In [219]:
import operator
max_lat_list = []
for v in range(V):
    #tuple (v, cache, lat_gain_value)
    max_lat_list.append((v, max(lat_gain[v].items(), key=operator.itemgetter(1))[0], max(lat_gain[v].items(), key=operator.itemgetter(1))[1]))

In [220]:
size_availability = [X for i in range(C)]

In [ ]:
final_result = [[] for i in range(C)]
for i in range(V*C):
    a = max(max_lat_list, key = operator.itemgetter(2))
    if sizes_videos[a[0]] <= size_availability[a[1]]:
        #final_result.append((a[1], a[0]))
        final_result[a[1]].append(a[0])
        size_availability[a[1]] = size_availability[a[1]] - sizes_videos[a[0]]
        del lat_gain[a[0]][a[1]]
        update_latency(a)
        max_lat_list[a[0]] = (a[0], max(lat_gain[a[0]].items(), key=operator.itemgetter(1))[0], max(lat_gain[a[0]].items(), key=operator.itemgetter(1))[1])
        

In [ ]:
final_result

In [ ]:
individualc=final_result
fitness_value = 0    
for i in range(0,len(requests)):  
    #req[i,1]: index of the endPoint in req i
    cachesEndp = endpoints[requests[i][1]][2]# Get caches connect to endPoint of req i
    lci = 0
    Ldi=0
    for ci,lc in cachesEndp:
        if  requests[i][0] in individualc[ci]: # if the video from req i is in one or more of the caches
            aux = lc # get lattency of cache_x-endPoint_i
            if aux < lci  or lci==0:
                lci = aux
                Ldi = endpoints[requests[i][1]][ 0 ] 
    fitness_value += (Ldi - lci)*requests[i][2]
fitness_value *= 1000/np.sum([x[2] for x in requests])

In [ ]:
fitness_value
